# Projectile Motion

This section will look at solving the projectile motion of a baseball. Without air resistance, the problem can be solved with elementary physics. With air resistance, however, we must resort to numerical methods. 

The equation of motion for the baseball is:

$$ \frac{dv}{dt} = \frac{1}{m}F_a(v) - g\hat y $$

The force of air resistance is a function of the velocity. The last term is the gravitational acceleration multiplied by the unit vector in the y-direction. The force from air resistance is often approximated as:

$$ F_a = -\frac{1}{2}C_d\rho A |v|v $$

The drag coefficient of a baseball is dependent on its Reynolds number. Its drag coefficient is also typically lower than that of a smooth sphere because the stitching disrupts laminar flow around the ball, precipitating the onset of turbulence. At low Reynolds numbers, the drag coefficient is dependent on the Re itself. At higher Re, vortex shedding maintains a nearly constant drag coefficient of about 0.5. At the critical Re (approximately $3x10^{5}$ for a smooth sphere), turbulent flow disrupts the low pressure region in the wake behind the sphere.

A baseball's mass and diameter are 0.145 kg and 7.4 cm.

## Forward Derivative

Performing the numerical analysis requires computation of the derivative. The derivative is defined as:

$$ f'(x) = \lim_{\tau \to 0} \frac{f(t + \tau) - f(t)}{\tau} $$

A special version of a Taylor expansion can be used where:

$$ f(t + \tau) = f(t) + \tau f'(t) + \frac{\tau^2}{2}f''(\zeta) $$

Where $\zeta$ is a value between $t$ and $t + \tau$. The above expression can be rewritten as:

$$ f'(t) = \frac{f(t + \tau) - f(t)}{\tau} - \frac{1}{2}\tau f''(\zeta) $$

The last term is the truncation error. If the term is kept, the expression for $f'(t)$ is exact. But, we can't compute that term, because we don't know what $\zeta$ is. But, we can bound the error, because we know the limits: $\zeta$ is somewhere between $t$ and $t + \tau$.

## Euler's Method

The equations of motion to be solved, generally, are:

$$\frac{dv}{dt} = a(r, v); \frac{dr}{dt} = v $$

In projectile motion, the acceleration is only a function of $v$ because of air resistance. In later sections, such as the motions of comets, acceleration will also be a function of position.

Using the forward derivative, the equations of motion are:

$$ \frac{v(t + \tau) - v(t)}{\tau} + O(\tau) = a(r(t), v(t)) $$

$$ \frac{r(t + \tau) - r(t)}{\tau} + O(\tau) = v(t) $$

Which, when rearranged, evaluate to:

$$ v(t + \tau) = v(t) + \tau a(r(t), v(t)) + O(\tau^2) $$

$$ r(t + \tau) = r(t) + \tau v(t) + O(\tau^2) $$

How is Euler's method implemented in practice?